<img src="header.png" align="left"/>

# Anwendungsbeispiel Incremental/active learning and optimization

Das Ziel dieses Beispiels ist die Erklärung von **incremental learning** und **active learning**. Dabei wird versucht aus neuen Daten weitere Trainingsdaten zu generieren mit denen ein Modell verbessert werden kann. Im zweiten Teil wird noch die Technik der **hyperparameter optimization** vorgestellt.


Der Code für die hyperparameter optimization wurde aus [1] entlehnt.

- [1] [https://medium.com/district-data-labs/parameter-tuning-with-hyperopt-faa86acdfdce](https://medium.com/district-data-labs/parameter-tuning-with-hyperopt-faa86acdfdce)


Referenz auf den EMNIST Datensatz:
```
EMNIST: Extending MNIST to handwritten letters, Cohen, Gregory and Afshar, Saeed and Tapson, Jonathan and Schaik, Andre Van, 2017 International Joint Conference on Neural Networks (IJCNN), 2017
```

# Import der Module  

In [ ]:
import numpy as np
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.datasets import mnist

In [ ]:
#
# Einstellen der Grösse von Diagrammen
#
plt.rcParams['figure.figsize'] = [16, 14]

# Incremental Learning

Bei incremental learning werden aus neuen Daten mit Hilfe von Regeln neue Trainingsdaten extrahiert und damit ein Modell neu trainiert. Wir werden sehen, ob das funktioniert.

Mehr zu incremental learning unter [https://en.wikipedia.org/wiki/Incremental_learning](https://en.wikipedia.org/wiki/Incremental_learning)

<img src="info.png" align="left"/> 


## Das alte Modell

In [ ]:
#
# Parameter für Modell
#
prefix = 'results/04_'
modelName = prefix + "model.json"
weightName = prefix + "model.h5"

In [ ]:
#
# Laden des vortrainierten Modelles aus Anwendungsbeispiel 04
#
json_file = open(modelName, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(weightName)
print("loaded model from disk")

In [ ]:
#
# Vorbereitung für Test
#
optimizer = Adam(learning_rate=0.001)
loaded_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#
# MNIST Daten (wurden für das Trainings des alten Modelles verwendet)
#
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
x_test.shape

In [ ]:
#
# Messen der Accuracy des alten Modelles
#
_, acc = loaded_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy {:.5f}'.format(acc))

In [ ]:
#
# Speichern für später
#
oldModelAccuracy = acc

In [ ]:
#confusion_matrix(y_test, y_pred)

# Neue Daten aus dem echten Betrieb

Wir simulieren neue Daten indem wir Daten aus einem anderen Datensatz EMNIST verwenden. Im Produktiveinsatz treten in der Regel immer neue Daten auf. Das Model wurde auf MNIST trainiert und muss jetzt die EMNIST Daten klassifieren. Diese Daten hat das Modell noch nie gesehen. Es sind also Überraschungen zu erwarten.

In [ ]:
#
# Laden der neuen EMNIST Daten
#
# y_new = np.loadtxt('data/emnist_labels.csv',dtype=np.int8,delimiter=",")
#
x_new = np.loadtxt('data/emnist_images.csv',delimiter=",")

In [ ]:
print(x_new.shape)

In [ ]:
x_new = np.reshape(x_new,(-1,28,28,1))

In [ ]:
print(x_new.shape)

In [ ]:
#
# Anzeige von Beispielen der Daten
#
for i in range(16):
    plt.subplot(4,4,1 + i)
    image = x_new[i].reshape((28,28))
    plt.imshow(image, cmap=plt.get_cmap('Greys_r'))
plt.show()

In [ ]:
#
# Aus einem unklaren Grund sind die EMNIST Daten gedreht. Das muss korrigiert werden.
#
x_new_swap = np.swapaxes(x_new, 1, 2)

In [ ]:
#
# Anzeige von Beispielen der Daten
#
for i in range(16):
    plt.subplot(4,4,1 + i)
    image = x_new_swap[i].reshape((28,28))
    plt.imshow(image, cmap=plt.get_cmap('Greys_r'))
plt.show()

In [ ]:
# 
# Transformation
#
x_new_swap = x_new_swap.astype('float32')
x_new_swap = x_new_swap / 255.0

In [ ]:
#
# Suche nach neuen Trainingsdaten
#
# Die Idee dahinter ist, dass wir nach Klassifizierungen suchen, die in einem bestimmten Bereich der confidence
# liegen. Zu hohe confidence steigert das Risiko, dass wir eh schon bekannte Daten dazunehmen, zu niedrige confidence
# steigert das Risiko, dass wir falsche Klassifizierungen dazunehmen.
#

candidateImages = []
candidateLabels = []
candidateConfidence = []
candidateIndex = []
candidateCount = 0
classDistribution = [0] * 10

for i in range(x_new_swap.shape[0]):
    
    image = x_new_swap[i].reshape((1,28,28,1))

    prediction_activation = loaded_model.predict([image])
    predictedClass = np.argmax ( prediction_activation[0] )
    confidence = prediction_activation[0][predictedClass]
    
    if confidence > 0.7 and confidence < 0.9:
        
        classDistribution[predictedClass]+= 1
        candidateCount+= 1
        candidateIndex.append(i)
        candidateImages.append(image)
        candidateLabels.append(predictedClass)
        candidateConfidence.append(confidence)

In [ ]:
print('Anzahl der gefundenen Kandidaten ist {}'.format(candidateCount))

In [ ]:
#
# Anzeige von Beispielen der Kandidaten
#
for i in range(16):
    ax = plt.subplot(4,4,1 + i) 
    ax.set_title('{} pred {} conf {:.2f}'.format ( candidateIndex[i], str(candidateLabels[i]), candidateConfidence[i] ))
    image = candidateImages[i].reshape((28,28))
    plt.imshow(image, cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
#
# Manuelle Auswahl der Kandidaten durch den Domain Experten (active learning)
#
candidatesUsed =       [ 77,110,130,142,210,240,310,363,365,454,445,412,367,212]
candidatesUsedLabels = [  2,  9,  9,  9,  2,  3,  7,  3,  9,  1,  9,  9,  8,  3]

In [ ]:
#
# Sammeln der Daten aus der Liste
#
candidateImagesUsed = []
candidateLabelsUsed = []
for candidate in range(len(candidatesUsed)):
    candidateLabelsUsed.append(candidatesUsedLabels[candidate])
    candidateImagesUsed.append(candidateImages[candidate])
    
x_train_used = np.asarray ( candidateImagesUsed )
y_train_used = np.asarray ( candidateLabelsUsed )


In [ ]:
#
# Prepare new data for training
#
x_train_used = x_train_used.reshape((x_train_used.shape[0], 28, 28, 1))
y_train_used = to_categorical(y_train_used, num_classes=10)

In [ ]:
#
# Zusammenführen der neuen und der alten Daten
#
x_train_active = np.concatenate((x_train_used, x_train), axis=0)
y_train_active = np.concatenate((y_train_used, y_train), axis=0)

In [ ]:
x_train_active.shape

In [ ]:
y_train_active.shape

In [ ]:
#
# Prepare Model for retraining
#
optimizer = Adam(learning_rate=0.01)
loaded_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#
# Training mit neuen Daten
#
history = loaded_model.fit(x_train_active, y_train_active, validation_data=(x_test,y_test), batch_size=64, epochs=1 )

In [ ]:
#
# Messen der Accuracy des alten Modelles
#
_, acc = loaded_model.evaluate(x_test, y_test, verbose=0)
print('New model accuracy {:.5f}'.format(acc))

In [ ]:
#
# Alte model accuracy zum Vergleich 
#
print('Old model accuracy {:.5f}'.format(oldModelAccuracy))

# Hyperparameter Optimization


Mehr zu hyperparameter optimization ist hier zu finden: 
- [https://towardsdatascience.com/hyperparameters-optimization-526348bb8e2d](https://towardsdatascience.com/hyperparameters-optimization-526348bb8e2d)
- [https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f)


<img src="info.png" align="left"/> 

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score

In [ ]:
#
# Vorbereitung der MNIST Daten für RFC
#
x_train_rfc = np.reshape(x_train,(-1,28*28))
x_train_rfc.shape

In [ ]:
y_train_rfc = np.argmax(y_train,axis=1)
y_train_rfc.shape

In [ ]:
x_test_rfc = np.reshape(x_test,(-1,28*28))
y_test_rfc = np.argmax(y_test,axis=1)

## Random Forest Classifier

[https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=20,max_depth=6,max_features=0.2,n_jobs=-1)
clf_rf.fit(x_train_rfc, y_train_rfc)

In [ ]:
y_pred_rf = clf_rf.predict(x_test_rfc)

In [ ]:
acc_rf = accuracy_score(y_test_rfc, y_pred_rf)
print ("random forest accuracy: ",acc_rf)

## Wie finden wir die optimalen Parameter?

Es gibt mehrere python Module für die automatische Parameter Optimierung. Eines davon ist hyperopt. Hintergrundinformation dazu ist zu finden unter: [https://conference.scipy.org/proceedings/scipy2013/pdfs/bergstra_hyperopt.pdf](https://conference.scipy.org/proceedings/scipy2013/pdfs/bergstra_hyperopt.pdf)


In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
import os

# um mehr als einen worker zu verwenden sollte JOBLIB_TEMP_FOLDER global gesetzt sein
os.environ["JOBLIB_TEMP_FOLDER"] = "./data"

In [ ]:
def hyperopt_train_test(params):
    clf = RandomForestClassifier( n_jobs=-1, **params)
    return cross_val_score(clf, x_train_rfc, y_train_rfc).mean()

In [ ]:
space4rf = {
    'max_depth': hp.choice('max_depth', range(1,20)),
    'max_features': hp.uniform('max_features', 0, 1 ),
    'n_estimators': hp.choice('n_estimators', range(1,30)),
    'criterion': hp.choice('criterion', ["gini", "entropy"])
}

In [ ]:
best = 0
def f(params):
    global best
    acc = hyperopt_train_test (params)
    if acc > best:
        best = acc
        print ( 'new best accuracy {}:{}'.format( best, params ) )
    return {'loss': -acc, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best = fmin(f, space4rf, algo=tpe.suggest, max_evals=20, trials=trials)

In [ ]:
print ('found optimum {}'.format(best))

In [ ]:
parameters = ['n_estimators', 'max_depth', 'max_features', 'criterion', 'scale', 'normalize']
f, axes = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
cmap = plt.cm.jet
for i, val in enumerate(parameters):
    print i, val
    xs = np.array([t['misc']['vals'][val] for t in trials.trials]).ravel()
    ys = [-t['result']['loss'] for t in trials.trials]
    xs, ys = zip(\*sorted(zip(xs, ys)))
    ys = np.array(ys)
    axes[i/3,i%3].scatter(xs, ys, s=20, linewidth=0.01, alpha=0.5, c=cmap(float(i)/len(parameters)))
    axes[i/3,i%3].set_title(val)
    #axes[i/3,i%3].set_ylim([0.9,1.0])

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# https://medium.com/@sebastiannorena/some-model-tuning-methods-bfef3e6544f0